Code mining partie PS 

In [ ]:
from pynq import Overlay

SHA256 = Overlay('/home/xilinx/jupyter_notebooks/DJ/dj.bit')

ip=SHA256.sha256_0
mmio=ip.mmio
register_map=ip.register_map
registers=register_map._register_classes

In the previous part, we create the overlay (material library) for our hash function (SHA 256) which is implemented in the FPGA and will be used by next

In [ ]:
# Show all registers available
for name, reg in registers.items():
    print(name, reg)

In [ ]:
import hashlib
import array
import math
import numpy as np
from pynq import allocate


# Hardware accelerated function
def SHA256_hardware(xxx, byte, base):
    
    from pynq import allocate
    
    register_map.bytes=byte
    register_map.base_offset=base

    # Register location 
    result_addr = register_map.Memory_digest.address // 4
    result_size = register_map.Memory_digest.width // 4
    result_val = mmio.array[result_addr:result_addr+result_size]
  
    input_buffer_size = 256
    input_buffer = allocate(shape=(input_buffer_size,), dtype='u1', cacheable=False) # unsigned 32-bit integers
    register_map.Memory_data = input_buffer.device_address
    
     from functools import reduce 

    def s2h (s):
        list=[]
        for ch in s :
            s2h=hex(ord(ch)).replace('0x', '')
            if len(s2h)==1 : s2h='0'+s2h
            list.append(s2h)
     
        return str(hex(int(reduce(lambda i, j:i+j,list),16))).encode()
         
    
    x=s2h(xxx)
    
    
     # Write to input buffer
    input_buffer[:len(x)] = bytearray(x)
    
    
    register_map.CTRL = 1
    
   
     #Wait until algorithm has completed
        
    while (register_map.CTRL.AP_DONE == 0):
        pass
    
    result = bytearray(result_val)
    return result.hex()


def SHA256_software(xxx):
    return hashlib.sha256(xxx).hexdigest()